# Hands-on KI mit PyTorch: MNIST (20 Minuten)

In diesem Notebook wirst du ein einfaches neuronales Netz auf dem MNIST-Datensatz trainieren. 
Wichtige Teile fehlen absichtlich – du wirst sie selbst implementieren.

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

## 1. MNIST-Datensatz laden
Wir laden Trainings- und Testdaten.

Es ist wichtig die seperate Trainings und Testdaten zu definieren um festzustellen ob ein overfitting stattfindet.
Overfitting bezeichnet ein auswendig Lernen der exakten Testdaten im Gegensatz zum Lernen von Mustern.
Der Testdatensatz überprüft daher, ob das Modell mit Daten umgehen kann welche es noch nie gesehen hat.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 1-2 Beispiele plotten
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)
plt.figure(figsize=(5,2))
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
    plt.title(f'Label: {example_targets[i]}')
    plt.axis('off')
plt.show()

## 2. Neuronales Netz definieren

✍️ **Aufgabe:** Implementiere ein einfaches fully connected Netzwerk.
- Wähle die Anzahl der Hidden Layer (1 oder 2)
- Wähle die Layer-Größen (z.B. 2, 16, 64, 128)
- Wähle die Aktivierungsfunktion (ReLU, Sigmoid, Tanh)

**Hinweis / Beispiel:**\
In der init Funktion musst du alle Layer und Aktivierungsfunktionen die du nutzen willst definieren.\
Achte darauf, dass du bei mehreren Layern (z.B. Linear) die Dimensionen für input und output entsprechend wählst.\
! Die letzte Layer sollte auf 10 Outputs reduzieren ! 
```python
self.fc1 = nn.Linear(28*28, 2)  # Layergröße ändern nach Wahl
self.fc2 = nn.Linear(2, 32) # Layer mit 128 inputs und 32 Outputs
self.act = nn.ReLU()  # Aktivierung wählen
```

Die forward Funktion reicht die Daten weiter zwischen den Layers
```python
data_after_layer_one = self.fc1(x)
data_after_activation_function = self.act(data_after_layer_one)
```

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # TODO: Definiere deine Layer hier
        pass

    def forward(self, x):
        # TODO: Implementiere forward pass
        pass

model = SimpleNN()

## 3. Training vorbereiten

✍️ **Aufgabe:** Wähle Loss-Funktion und Optimizer
- Loss: CrossEntropyLoss, MSELoss (Mean-Squared-Error), L1Loss (Mean-Absolute-Error), ... [more](https://neptune.ai/blog/pytorch-loss-functions)
- Optimizer: SGD oder Adam

**Hinweis / Beispiel:**
```python
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
```

In [ ]:
# TODO: Loss & Optimizer definieren
criterion = None  # setze Loss-Funktion
optimizer = None  # setze Optimizer

## 4. Trainingsloop

✍️ **Aufgabe:** Implementiere den Trainingsloop für z.B. 3 Epochen

**Hinweis / Beispiel:**
```python
for epoch in range(3):
    for data, target in train_loader:
        optimizer.zero_grad()                       # zurücksetzen aller Gradienten
        output = model(data.view(data.size(0), -1)) # Bilder werden als Vektoren in das Modell gegeben
        loss = criterion(output, target)            # Loss/Fehler des Models wird berechnet
        loss.backward()                             
        # berechnet Gradienten für alle Gewichte/Parameter in Layern
        # quasi wie stark der Parameter geändert werden muss damit eher das passende Ergebnis erreicht wird 
        optimizer.step()                            # Anpassen der Gewichte abhängig von Gradient und learning rate

```

oft ist es interessant den loss über die Zeit aufzuzeichnen. Um das zu erreichen kannst du die Loss Werte aufzeichnen

```python
loss_array = []

*training_loop*
    loss_array.append(loss.item())

plt.plot(loss_array)
```

In [ ]:
# TODO: Trainingsloop einfügen
pass

## 5. Evaluation

Berechne Accuracy auf Testdaten.



In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data.view(data.size(0), -1))
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)
accuracy = correct / total
print(f'Accuracy: {accuracy*100:.2f}%')

## 6. Ergebnisse visualisieren
Zeige einige Testbilder mit Vorhersagen des Modells.

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
output = model(example_data.view(example_data.size(0), -1))
preds = output.argmax(dim=1)
plt.figure(figsize=(10,4))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
    plt.title(f'True: {example_targets[i]} Pred: {preds[i]}')
    plt.axis('off')
plt.show()

Außerdem schauen wir uns einmal die falsch klassifizierten Zahlen an

In [ ]:
number_of_misclassified_to_show = 5
misclassified_found = 1
with torch.no_grad():
    for data, target in test_loader:
        output = model(data.view(data.size(0), -1))
        pred = output.argmax(dim=1)

        for i in range(len(data)):
            if pred[i] != target[i]:
                print(f"True Label: {target[i].item()}")
                print(f"Predicted Label: {pred[i].item()}")
                print(f"Model Outputs (logits): {output[i].numpy()}")

                plt.figure(figsize=(3,3))
                plt.imshow(data[i][0], cmap='gray')
                plt.title(f'True: {target[i].item()} Pred: {pred[i].item()}')
                plt.axis('off')
                plt.show()
                misclassified_found +=1
                if misclassified_found > number_of_misclassified_to_show:
                  break
        if misclassified_found>number_of_misclassified_to_show:
            break

if not misclassified_found:
    print("No misclassified examples found in the first batch (or no misclassifications at all).")

<!-- # 7. Spielereien
Teste auf einem trainierten Modell ob es deine Zahlen erkennt: [[https://interactive-digit-classifier.onrender.com/](https://interactive-digit-classifier.onrender.com/)]

Wie reagiert es auf Buchstaben oder andere Symbole? -->

# 7. Spielereien
Teste deine eigene Handschrift auf einem trainierten Modell: [interactive-digit-classifier](https://interactive-digit-classifier.onrender.com/) [cnn-playground](https://cnn-playground.live/mnist)

Wie reagiert das Modell auf andere Symbole?

